# Distributed K-Means Testing

## Imports and basic functions

#### Imports libs needed and created function to get available GPUs and to make training data

In [ ]:
import matplotlib.pyplot as plt

import threading

import os
import time

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.client import device_lib

from sklearn.datasets import make_classification

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def make_data(filepath, n_obs, n_dim, seed, K):

    try:
        os.remove(filepath)
    except:
        print('file not found')
    finally:
        (X, Y) = make_classification(n_samples            = n_obs    , 
                                     n_features           = n_dim    ,
                                     n_informative        = n_dim    ,
                                     n_redundant          = 0        ,
                                     n_classes            = K        ,
                                     n_clusters_per_class = 1        ,
                                     shuffle              = True     ,
                                     class_sep            = 1.5      ,
                                     random_state         = seed      )
        
        np.savez(filepath, X=X, Y=Y)

    return True

#### Creates a Tensorflow InteractiveSession

An InteractiveSession is used to "debug" the code. It allows you to see what's happening. On production code, use Session instead.

In [ ]:
# 

#### Defining variables used throughout the code

In [ ]:
n_obs = 200000000
n_dim = 2
K     = 3
GPU_names = get_available_gpus()
n_max_iters = 20
seed = 800594

In [ ]:
make_data('test-data.npz', n_obs, n_dim, seed, K)

In [ ]:
data = np.load('test-data.npz')

data_X = data['X']
data_Y = data['Y']
initial_centers = data_X[0:K, :]

#### Divide dataset into X equal parts, X being the number of available GPUs

In [ ]:
sizes = [len(arg) for arg in np.split( data_X[ (data_X.shape[0] % len(GPU_names)) :, :], len(GPU_names))]

partial_directions = []
partial_values = []
partial_results = []

In [ ]:
data_size = n_dim * n_obs * data_X.dtype.itemsize
max_size = 2*1024*1024*1024

num_batches = np.ceil(data_size / max_size)

batches = np.array_split(data_X, num_batches)

print(batches)

#### Creates Tensorflow variable with the initial centroids

In [ ]:
with tf.name_scope('global'):
    with tf.device('/cpu:0'):
        all_data = tf.placeholder(data_X.dtype, shape=(data_X.shape), name='all_data')
        print(all_data)

        parts = tf.split(tf.Variable(all_data), sizes, 0)
        print(tf.Variable(parts[0]))
        
        global_centroids = tf.Variable(initial_centers)

#### For each GPU, calculate new local centroids

In [ ]:
for GPU_num in range(len(GPU_names)):
    GPU_name = GPU_names[GPU_num]
    
    (X_mat) = parts[GPU_num]
    (N, M) = X_mat.get_shape().as_list()
    
    with tf.name_scope('scope_' + str(GPU_num)):
        with tf.device(GPU_name) :
            ####
            # In the coments we denote :
            # => N = Number of Observations
            # => M = Number of Dimensions
            # => K = Number of Centers
            ####
            
            # Data for GPU GPU_num to Clusterize            
            X = tf.Variable(X_mat)

            # Reshapes rep_centroids and rep_points to format N x K x M so that 
            # the 2 matrixes have the same size
            rep_centroids = tf.reshape(tf.tile(global_centroids, [N, 1]), [N, K, M])
            rep_points = tf.reshape(tf.tile(X, [1, K]), [N, K, M])

            # Calculates sum_squares, a matrix of size N x K
            # This matrix is not sqrt((X-Y)^2), it is just(X-Y)^2
            # Since we need just the argmin(sqrt((X-Y)^2)) wich is equal to 
            # argmin((X-Y)^2), it would be a waste of computation
            sum_squares = tf.reduce_sum(tf.square(tf.subtract( rep_points, rep_centroids) ), axis = 2)

            # Use argmin to select the lowest-distance point
            # This gets a matrix of size N x 1
            best_centroids = tf.argmin(sum_squares, axis = 1)
            
            means = []
            for c in range(K):
                means.append(
                    tf.reduce_mean(
                        tf.gather(X, tf.reshape(tf.where(tf.equal(best_centroids, c)), [1,-1])), axis=[1]))

            new_centroids = tf.concat(means, 0)
#             print('GPU: ', GPU_name)
#             print('Initial centers ', initial_centers)
#             print('New centroids ', new_centroids.eval())
                
        with tf.device('/cpu:0'):
            y_count = tf.cast(
                tf.bincount(tf.to_int32(best_centroids), maxlength = K, minlength = K), dtype = tf.float64)
            
            partial_mu =  tf.multiply( tf.transpose(new_centroids), y_count )

            partial_directions.append( y_count )
            partial_values.append( partial_mu )

#### After local GPU centroids have been calculated, create new global centroids

In [ ]:
with tf.name_scope('global') :
    with tf.device('/cpu:0') :
        sum_direction = tf.add_n( partial_directions )
        sum_mu = tf.add_n( partial_values )

        rep_sum_direction = tf.reshape(tf.tile(sum_direction, [M]), [M, K])
        new_centers = tf.transpose( tf.div(sum_mu, rep_sum_direction) )

        update_centroid = tf.group( global_centroids.assign(new_centers) )

In [ ]:
config = tf.ConfigProto( log_device_placement = True, allow_soft_placement = True )
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

init = tf.global_variables_initializer()

print(data_X.dtype, data_X.shape)

In [ ]:
with tf.Session(config = config) as sess:
    sess.run(init, feed_dict={all_data: data_X})
    
    for i in range(n_max_iters):
        [result, _] = sess.run([global_centroids, update_centroid])

### Plotting graphs to evaluate results

#### Initial centroids

In [ ]:
plt.scatter(data_X[0:np.min([data_X.shape[0], 10000]), 0], 
            data_X[0:np.min([data_X.shape[0], 10000]), 1], 
            c = data_Y[0:np.min([data_X.shape[0], 10000])],
            alpha = 0.8, marker = (5, 2))
plt.scatter(initial_centers[:, 0], initial_centers[:, 1], alpha = 1, c = 'red', marker = (5, 3))
plt.show()

#### After computation

In [ ]:
plt.scatter(data_X[0:np.min([data_X.shape[0], 10000]), 0], 
            data_X[0:np.min([data_X.shape[0], 10000]), 1], 
            c = data_Y[0:np.min([data_X.shape[0], 10000])],
            alpha = 0.8, marker = (3, 1))
plt.scatter(result[:, 0], result[:, 1], alpha = 1, c = 'blue', marker = (5, 3))
plt.show()